In [16]:
import math
from datetime import timedelta

import numpy as np
import pandas as pd
from tqdm import tqdm

pd.set_option('display.max_columns', None)

# Version 1

In [20]:
tweets_df = pd.read_csv("all_tweets.csv", index_col=0)
tweets_df.head(3)

/var/folders/z5/hjyvlbm54v1cpt9s019vbv9w0000gn/T/ipykernel_1139/971841145.py:1: DtypeWarning: Columns (15,26) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets_df = pd.read_csv("all_tweets.csv", index_col=0)


,author_id,context_annotations,conversation_id,created_at,edit_history_tweet_ids,id,in_reply_to_user_id,lang,possibly_sensitive,referenced_tweets,reply_settings,text,author,withheld,geo,quote_count,retweet_count,like_count,reply_count,impression_count,media_keys,poll_ids,hashtags,urls,mentions,cashtags,annotations,is_retweet
0,1449804331142811655,NaN,1617597872803041280,2023-01-23 18:59:21+00:00,[1617597872803041280],1617597872803041280,NaN,tr,False,NaN,everyone,"Kararlarındaki temel dayanak, hukukun evrensel...",Goksun_KHK,NaN,NaN,0,2,1,0,82,NaN,NaN,"[{'start': 95, 'end': 120, 'tag': 'OhalKomisyo...",NaN,NaN,NaN,NaN,False
1,1449804331142811655,NaN,1617597646339702823,2023-01-23 18:58:27+00:00,[1617597646339702823],1617597646339702823,NaN,qme,False,NaN,everyone,#OhalKomisyonuHukuksuzdur https://t.co/3aUr5MZWPU,Goksun_KHK,NaN,NaN,0,1,0,0,52,['3_1617597636604723228'],NaN,"[{'start': 0, 'end': 25, 'tag': 'OhalKomisyonu...","[{'start': 26, 'end': 49, 'url': 'https://t.co...",NaN,NaN,NaN,False
2,1449804331142811655,NaN,1617597571903676417,2023-01-23 18:58:09+00:00,[1617597571903676417],1617597571903676417,NaN,tr,False,[<ReferencedTweet id=1617584897383714816 type=...,everyone,RT @Yozgat_KHK: #OhalKomisyonuHukuksuzdur\nÇün...,Goksun_KHK,NaN,NaN,0,48,0,0,0,NaN,NaN,"[{'start': 16, 'end': 41, 'tag': 'OhalKomisyon...",NaN,"[{'start': 3, 'end': 14, 'username': 'Yozgat_K...",NaN,NaN,True


# Version 2

## What is done

1. Gender of the author is added to each tweet
2. Profession of the author is added to each tweet
3. Hashtags are extracted from a dictionary values and joined with comma into `hashtags_flattened` column
4. Existence of trend topics are extracted and two related columns are added.

In [21]:
users_df = pd.read_excel("HRA Twitter Accounts.xlsx")
users_df = users_df[users_df.valid == 1]
users_df.head(3)

/var/folders/z5/hjyvlbm54v1cpt9s019vbv9w0000gn/T/ipykernel_1139/2019825818.py:1: FutureWarning: Inferring datetime64[ns, UTC] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns, UTC])
  users_df = pd.read_excel("HRA Twitter Accounts.xlsx")


,full_name,screen_name,method_to_find,comment,valid,location,name,created_at,description,verified,followers_count,following_count,tweet_count,id,gender,profession,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,Akın Birdal,akin_birdal,web-search,NaN,1,NaN,Akın Birdal,2013-04-14 16:39:04+00:00,NaN,False,80216.0,786.0,2937.0,1.352279e+09,M,UNKNOWN,NaN,NaN,NaN
2,Cemil Çiçek,avcemilcicek,well-known,NaN,1,"Çankaya, Ankara",Cemil Çiçek,2011-07-20 11:49:01+00:00,Hukukçu/Law - @ASBUedu Asian Studies MA | Ceza...,False,112011.0,1361.0,14618.0,3.389775e+08,M,LAWYER,NaN,NaN,NaN
4,Ayşe Gül Altınay,aysealtinay,web-search,NaN,1,Istanbul,Ayşe Gül Altınay,2010-11-28 23:09:32+00:00,NaN,False,6883.0,2519.0,10187.0,2.208243e+08,F,UNKNOWN,NaN,NaN,NaN


In [22]:
# Gender and Profession is added
tweets_df_v2 = pd.merge(tweets_df, users_df[["screen_name", "gender", "profession"]],  left_on="author", right_on="screen_name")
tweets_df_v2.created_at = pd.to_datetime(tweets_df_v2.created_at, format='%Y-%m-%d %H:%M:%S+00:00') 
tweets_df_v2.drop(columns=["screen_name"], inplace=True)
tweets_df_v2.head(3)

,author_id,context_annotations,conversation_id,created_at,edit_history_tweet_ids,id,in_reply_to_user_id,lang,possibly_sensitive,referenced_tweets,reply_settings,text,author,withheld,geo,quote_count,retweet_count,like_count,reply_count,impression_count,media_keys,poll_ids,hashtags,urls,mentions,cashtags,annotations,is_retweet,gender,profession
0,1449804331142811655,NaN,1617597872803041280,2023-01-23 18:59:21,[1617597872803041280],1617597872803041280,NaN,tr,False,NaN,everyone,"Kararlarındaki temel dayanak, hukukun evrensel...",Goksun_KHK,NaN,NaN,0,2,1,0,82,NaN,NaN,"[{'start': 95, 'end': 120, 'tag': 'OhalKomisyo...",NaN,NaN,NaN,NaN,False,ORG,NOT AVAILABLE
1,1449804331142811655,NaN,1617597646339702823,2023-01-23 18:58:27,[1617597646339702823],1617597646339702823,NaN,qme,False,NaN,everyone,#OhalKomisyonuHukuksuzdur https://t.co/3aUr5MZWPU,Goksun_KHK,NaN,NaN,0,1,0,0,52,['3_1617597636604723228'],NaN,"[{'start': 0, 'end': 25, 'tag': 'OhalKomisyonu...","[{'start': 26, 'end': 49, 'url': 'https://t.co...",NaN,NaN,NaN,False,ORG,NOT AVAILABLE
2,1449804331142811655,NaN,1617597571903676417,2023-01-23 18:58:09,[1617597571903676417],1617597571903676417,NaN,tr,False,[<ReferencedTweet id=1617584897383714816 type=...,everyone,RT @Yozgat_KHK: #OhalKomisyonuHukuksuzdur\nÇün...,Goksun_KHK,NaN,NaN,0,48,0,0,0,NaN,NaN,"[{'start': 16, 'end': 41, 'tag': 'OhalKomisyon...",NaN,"[{'start': 3, 'end': 14, 'username': 'Yozgat_K...",NaN,NaN,True,ORG,NOT AVAILABLE


In [23]:
# adding a column with all hashtags as strings
lst_hashtags = []

for i in tqdm(tweets_df_v2['hashtags']):
    if type(i) == float and math.isnan(i):
        lst_hashtags.append("")
    else:
        hashtags = ", ".join([j['tag'] for j in eval(i)])
        lst_hashtags.append(hashtags)
    
tweets_df_v2['hashtags_flattened'] = lst_hashtags
tweets_df_v2.head(2)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 359036/359036 [00:00<00:00, 376713.67it/s]


,author_id,context_annotations,conversation_id,created_at,edit_history_tweet_ids,id,in_reply_to_user_id,lang,possibly_sensitive,referenced_tweets,reply_settings,text,author,withheld,geo,quote_count,retweet_count,like_count,reply_count,impression_count,media_keys,poll_ids,hashtags,urls,mentions,cashtags,annotations,is_retweet,gender,profession,hashtags_flattened
0,1449804331142811655,NaN,1617597872803041280,2023-01-23 18:59:21,[1617597872803041280],1617597872803041280,NaN,tr,False,NaN,everyone,"Kararlarındaki temel dayanak, hukukun evrensel...",Goksun_KHK,NaN,NaN,0,2,1,0,82,NaN,NaN,"[{'start': 95, 'end': 120, 'tag': 'OhalKomisyo...",NaN,NaN,NaN,NaN,False,ORG,NOT AVAILABLE,OhalKomisyonuHukuksuzdur
1,1449804331142811655,NaN,1617597646339702823,2023-01-23 18:58:27,[1617597646339702823],1617597646339702823,NaN,qme,False,NaN,everyone,#OhalKomisyonuHukuksuzdur https://t.co/3aUr5MZWPU,Goksun_KHK,NaN,NaN,0,1,0,0,52,['3_1617597636604723228'],NaN,"[{'start': 0, 'end': 25, 'tag': 'OhalKomisyonu...","[{'start': 26, 'end': 49, 'url': 'https://t.co...",NaN,NaN,NaN,False,ORG,NOT AVAILABLE,OhalKomisyonuHukuksuzdur


In [24]:
df_all_trends = pd.read_csv('df_all_trends.csv')
df_all_trends['date'] = pd.to_datetime(df_all_trends['date'], format='%d-%m-%Y')
df_all_trends['trend_topic'] = df_all_trends['trend_topic'].str.replace('#', '')
df_all_trends.head()

,time,trend_topic,date
0,00:10,Zamlar,2022-01-01
1,00:10,2022NewYear,2022-01-01
2,00:10,elektrik,2022-01-01
3,00:10,BurasıislamDevleti,2022-01-01
4,00:10,millipiyango,2022-01-01


In [25]:
df_all_trends = df_all_trends[['date', 'trend_topic']].drop_duplicates()
df_all_trends.head(3)

,date,trend_topic
0,2022-01-01,Zamlar
1,2022-01-01,2022NewYear
2,2022-01-01,elektrik


In [26]:
tweets_df_v2.head(2)

,author_id,context_annotations,conversation_id,created_at,edit_history_tweet_ids,id,in_reply_to_user_id,lang,possibly_sensitive,referenced_tweets,reply_settings,text,author,withheld,geo,quote_count,retweet_count,like_count,reply_count,impression_count,media_keys,poll_ids,hashtags,urls,mentions,cashtags,annotations,is_retweet,gender,profession,hashtags_flattened
0,1449804331142811655,NaN,1617597872803041280,2023-01-23 18:59:21,[1617597872803041280],1617597872803041280,NaN,tr,False,NaN,everyone,"Kararlarındaki temel dayanak, hukukun evrensel...",Goksun_KHK,NaN,NaN,0,2,1,0,82,NaN,NaN,"[{'start': 95, 'end': 120, 'tag': 'OhalKomisyo...",NaN,NaN,NaN,NaN,False,ORG,NOT AVAILABLE,OhalKomisyonuHukuksuzdur
1,1449804331142811655,NaN,1617597646339702823,2023-01-23 18:58:27,[1617597646339702823],1617597646339702823,NaN,qme,False,NaN,everyone,#OhalKomisyonuHukuksuzdur https://t.co/3aUr5MZWPU,Goksun_KHK,NaN,NaN,0,1,0,0,52,['3_1617597636604723228'],NaN,"[{'start': 0, 'end': 25, 'tag': 'OhalKomisyonu...","[{'start': 26, 'end': 49, 'url': 'https://t.co...",NaN,NaN,NaN,False,ORG,NOT AVAILABLE,OhalKomisyonuHukuksuzdur


In [27]:
def generate_trend_topic_columns(row):
    trend_topics_in_two_weeks = df_all_trends[df_all_trends.date.isin([(row.created_at - timedelta(days=i - 7)).date() for i in range(15)])].trend_topic.to_list()
    trend_topics_in_tweet = [hashtag for hashtag in row.hashtags_flattened.split(", ") if hashtag in trend_topics_in_two_weeks]
    return pd.Series({"trend_topics_in_tweet": ", ".join(trend_topics_in_tweet),  "n_trend_topics": len(trend_topics_in_tweet)})

tweets_df_v2[['trend_topics', 'n_trend_topics']] = tweets_df_v2.apply(generate_trend_topic_columns, axis=1)

In [28]:
tweets_df_v2.to_csv("all_tweets_v2.csv", line_terminator='\r\n')

/var/folders/z5/hjyvlbm54v1cpt9s019vbv9w0000gn/T/ipykernel_1139/540208934.py:1: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  tweets_df_v2.to_csv("all_tweets_v2.csv", line_terminator='\r\n')


# Version 3

## What is done

1. Feature Engineering related to replies, retweets, existence of the media etc.

In [32]:
tweets_df_v3 = pd.read_csv("all_tweets_v2.csv", index_col=0, low_memory=False)

In [41]:
tweets_df_v3['text_length'] = tweets_df_v3['text'].apply(lambda x: len(x) if type(x) is str else np.nan)
tweets_df_v3['has_media'] = np.where(tweets_df_v3['media_keys'].isna(), 0, 1)
tweets_df_v3['has_hashtags'] = np.where(tweets_df_v3['hashtags'].isna(), 0, 1)
tweets_df_v3['has_mentions'] = np.where(tweets_df_v3['mentions'].isna(), 0, 1)
tweets_df_v3['is_retweet'] = tweets_df_v3['is_retweet'].astype(int)
tweets_df_v3['is_reply'] = np.where(tweets_df_v3['in_reply_to_user_id'].isna(), 0, 1)
tweets_df_v3.rename(columns={"gender": "gender_of_author", "profession": "profession_of_author"}, inplace=True)
tweets_df_v3['tweet_type'] = np.where(tweets_df_v3['is_reply']==1, 'only_reply',
                      np.where(tweets_df_v3['is_retweet']==1, 'only_retweet', 'no_reply_and_no_retweet'))
tweets_df_v3['ratio_like'] = tweets_df_v3['like_count'] / tweets_df_v3['impression_count']
tweets_df_v3['ratio_retweet'] = tweets_df_v3['retweet_count'] / tweets_df_v3['impression_count']
tweets_df_v3.head(2)

,author_id,context_annotations,conversation_id,created_at,edit_history_tweet_ids,id,in_reply_to_user_id,lang,possibly_sensitive,referenced_tweets,reply_settings,text,author,withheld,geo,quote_count,retweet_count,like_count,reply_count,impression_count,media_keys,poll_ids,hashtags,urls,mentions,cashtags,annotations,is_retweet,gender_of_author,profession_of_author,hashtags_flattened,trend_topics,n_trend_topics,text_length,has_media,has_hashtags,has_mentions,is_reply,tweet_type,ratio_like,ratio_retweet
0,1449804331142811655,NaN,1617597872803041280,2023-01-23 18:59:21,[1617597872803041280],1617597872803041280,NaN,tr,False,NaN,everyone,"Kararlarındaki temel dayanak, hukukun evrensel...",Goksun_KHK,NaN,NaN,0,2,1,0,82,NaN,NaN,"[{'start': 95, 'end': 120, 'tag': 'OhalKomisyo...",NaN,NaN,NaN,NaN,0,ORG,NOT AVAILABLE,OhalKomisyonuHukuksuzdur,OhalKomisyonuHukuksuzdur,1,120,0,1,0,0,no_reply_and_no_retweet,0.012195,0.024390
1,1449804331142811655,NaN,1617597646339702823,2023-01-23 18:58:27,[1617597646339702823],1617597646339702823,NaN,qme,False,NaN,everyone,#OhalKomisyonuHukuksuzdur https://t.co/3aUr5MZWPU,Goksun_KHK,NaN,NaN,0,1,0,0,52,['3_1617597636604723228'],NaN,"[{'start': 0, 'end': 25, 'tag': 'OhalKomisyonu...","[{'start': 26, 'end': 49, 'url': 'https://t.co...",NaN,NaN,NaN,0,ORG,NOT AVAILABLE,OhalKomisyonuHukuksuzdur,OhalKomisyonuHukuksuzdur,1,49,1,1,0,0,no_reply_and_no_retweet,0.000000,0.019231


tweets_df_v2.to_csv("all_tweets_v2.csv", line_terminator='\r\n')